In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',index_col='Id')

In [3]:
train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [4]:
def fill_missing_values(df):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    for column in list(missing.index):
        if df[column].dtype == 'object':
            df[column].fillna(df[column].value_counts().index[0], inplace=True)
        elif df[column].dtype == 'int64' or 'float64' or 'int16' or 'float16':
            df[column].fillna(df[column].median(), inplace=True)


In [5]:
fill_missing_values(train)
fill_missing_values(test)
train.isnull().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 80, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
def impute_cats(df):
    object_cols = list(df.select_dtypes(exclude=[np.number]).columns)
    object_cols_ind = []
    for col in object_cols:
        object_cols_ind.append(df.columns.get_loc(col))

    label_enc = LabelEncoder()
    for i in object_cols_ind:
        df.iloc[:,i] = label_enc.fit_transform(df.iloc[:,i])

In [7]:
impute_cats(train)
impute_cats(test)

In [8]:
X = train.drop('SalePrice', axis=1)
y = np.ravel(np.array(train[['SalePrice']]))
print(y.shape)

(1460,)


In [9]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=1000,
                                      max_depth=15,
                                      min_samples_split=5,
                                      min_samples_leaf=5,
                                     )

kf = KFold(n_splits=5)
y_pred = cross_val_score(random_forest, X, y, cv=kf, n_jobs=-1)
y_pred.mean()

0.848688265353237

In [11]:
random_forest.fit(X, y)
rf_pred = random_forest.predict(test)

In [12]:
import xgboost as xgb
model = xgb.XGBRegressor()
model.fit(X,y)

[12:38:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [13]:
p = model.predict(test)

In [14]:
submission1 = pd.DataFrame()

submission1['Id'] = np.array(test.index)
submission1['SalePrice'] = p

In [15]:
submission1.to_csv('submission1.csv', index=False)
submission1 = pd.read_csv('submission1.csv')

In [16]:
submission = pd.DataFrame()

submission['Id'] = np.array(test.index)
submission['SalePrice'] = rf_pred

In [17]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv('submission.csv')